# Singly Diagonally Implicit Runge Kutta Order 4 


\begin{align}
K_1 = (M + A\Delta t/4)^{-1}(f_{t+ \Delta t/4} - A u_j)\\
K_2 = (M + A\Delta t/4)^{-1}(f_{t+3\Delta t/4} - A(u_j + \Delta t/2K_1))\\
K_3 = (M + A\Delta t/4)^{-1}(f_{t+11\Delta t/20} - A(u_j + \Delta t(17/15K_1-1/25K_2))\\
K_4 = (M + A\Delta t/4)^{-1}(f_{t+\Delta t/22} - A(u_j + \Delta t(371/1360K_1-137/2720K_2 + 15/544K_3))\\
u_{j+1} = u_j + \Delta t(25/4K_{1} - 49/48K_{2} + 125/16K_{3}-85/12K_{4})
\end{align}



https://en.wikipedia.org/wiki/List_of_Runge%E2%80%93Kutta_methods

In [1]:
from ngsolve import *
from math import pi
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [2]:
geo = SplineGeometry()
geo.AddRectangle( (-1, -1), (1, 1), 
                 bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.25))
# Draw(mesh)
fes = H1(mesh, order=3, dirichlet="bottom|right|left|top")

u,v = fes.TnT()

time = 0.0
dt = 0.001

In [3]:
b = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
# Draw(b,mesh,"wind")
from ngsolve.internal import visoptions
visoptions.scalfunction = "wind:0"

In [4]:
a = BilinearForm(fes, symmetric=False)
a += 0.01*grad(u)*grad(v)*dx + b*grad(u)*v*dx
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += u*v*dx
m.Assemble()

In [5]:
gfu = GridFunction(fes)

In [6]:
mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + dt/4*a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

In [7]:
t = Parameter(0.0)

In [8]:
omega=1
gausspt = exp(-6*((x+sin(omega*t))*(x+sin(omega*t))+y*y))-exp(-6*((x-sin(omega*t))*(x-sin(omega*t))+y*y))
# scene = Draw(gausspt,mesh,"ft",order=3)
# time = 0.0
# from time import sleep
# while time < 10 - 0.5 * dt:
#     t.Set(time)
#     scene.Redraw()
#     time += 1e-3
#     sleep(1e-6)

In [9]:
ft = LinearForm(fes)
ft += gausspt*v*dx
time = 0.0
t.Set(0.0)
gfu.Set((1-y*y)*x)
#gfu.Set(CoefficientFunction(0))
scene = Draw(gfu,mesh,"u")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [10]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    ft.Assemble()
    k_1 = invmstar*(ft.vec - a.mat*gfu.vec)
    k_2 = invmstar*(ft.vec-a.mat*(gfu.vec + dt/2*k_1))
    k_3 = invmstar*(ft.vec-a.mat*(gfu.vec + dt*(17/15*k_1 - 1/25*k_2)))
    k_4 = invmstar*(ft.vec-a.mat*(gfu.vec+dt*(371/1360*k_1 - 137/2720*k_2 + 15/544*k_3)))
    k_5 = invmstar*(ft.vec-a.mat*(gfu.vec+dt*(25/4*k_1 - 49/48*k_2 + 125/16*k_3 - 85/12*k_4)))
    gfu.vec.data = gfu.vec + dt*(25/4*k_1 -49/48*k_2 + 125/16*k_3 -85/12*k_4 + 1/4*k_5)
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    scene.Redraw()
print("")
time+=t_intermediate

 9.999999999999897676
